In [5]:
!pip install catboost

     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [57]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score,KFold,train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV

In [35]:
df = pd.read_csv('drive/MyDrive/data.csv')

In [36]:
df

,No,Date,Ticker,Category,Value,Timestamp,Close,Returns
0,1,38028.0,CWT,Utilities,0.934181,1.076454e+09,14.720000,start
1,2,38028.0,BLL,Process Industries,0.922862,1.076454e+09,8.095000,start
2,3,38028.0,APA,Energy Minerals,0.912117,1.076454e+09,39.830002,start
3,4,38028.0,MATX,Transportation,0.866946,1.076454e+09,16.394106,start
4,5,38028.0,ROST,Retail Trade,0.864789,1.076454e+09,7.712500,start
...,...,...,...,...,...,...,...,...
26420,26421,44601.0,SLGN,Process Industries,0.709506,1.644361e+09,43.750000,0.037731511
26421,26422,44601.0,PEP,Consumer Non-Durables,0.701507,1.644361e+09,172.020004,0.014580879
26422,26423,44601.0,SSNC,Technology Services,0.701123,1.644361e+09,80.730003,0.055258505
26423,26424,44601.0,GEF,Process Industries,0.697954,1.644361e+09,57.880001,0.450160328


In [37]:
df = df.drop(['No', 'Date', 'Timestamp'], axis=1)

In [38]:
df = df.dropna()

In [39]:
df = df[~(df['Returns'] == 'start')]
df = df[~(df['Returns'] == 'Start')]

In [40]:
correlation_matrix = df.corr()

In [41]:
y = df['Returns']
X = df.drop('Returns', axis=1)

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [43]:
cat_features = ['Category', 'Ticker']
params = {
    'n_estimators': [500, 1000],
    'learning_rate': [0.05, 0.01],
    'max_depth': [6, 8, 10],
    'eval_metric': ['RMSE']
}

In [44]:
param = {
    'n_estimators': 1000,
    'learning_rate': 0.01,
    'max_depth': 8,
    'eval_metric': 'RMSE'
}

In [45]:
model = CatBoostRegressor(**param)

In [46]:
model.fit(X_train, y_train, cat_features=cat_features, verbose=1000)

0:	learn: 0.1303737	total: 17.8ms	remaining: 17.8s
999:	learn: 0.1224471	total: 12.5s	remaining: 0us


In [47]:
y_pred = model.predict(X_test)

In [51]:
rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print('Testing performance')
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2))
print('MAE: {:.2f}'.format(mae))

Testing performance
RMSE: 0.13
R2: 0.02
MAE: 0.06


In [53]:
model = CatBoostRegressor(cat_features=cat_features)

In [58]:
kf=KFold(n_splits=5)
model_cv = GridSearchCV(estimator = model, 
                        param_grid = params, 
                        scoring= 'r2', 
                        cv = kf, 
                        verbose = 1000,
                        return_train_score=True)  

In [59]:
model_cv.fit(X_train, y_train) 

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
7:	learn: 0.1307758	total: 174ms	remaining: 21.6s
8:	learn: 0.1307387	total: 199ms	remaining: 21.9s
9:	learn: 0.1307088	total: 223ms	remaining: 22.1s
10:	learn: 0.1306663	total: 245ms	remaining: 22s
11:	learn: 0.1306337	total: 270ms	remaining: 22.2s
12:	learn: 0.1306222	total: 275ms	remaining: 20.9s
13:	learn: 0.1305867	total: 315ms	remaining: 22.2s
14:	learn: 0.1305754	total: 321ms	remaining: 21.1s
15:	learn: 0.1305494	total: 345ms	remaining: 21.2s
16:	learn: 0.1305263	total: 369ms	remaining: 21.4s
17:	learn: 0.1305091	total: 391ms	remaining: 21.3s
18:	learn: 0.1304864	total: 418ms	remaining: 21.6s
19:	learn: 0.1304629	total: 442ms	remaining: 21.7s
20:	learn: 0.1304298	total: 465ms	remaining: 21.7s
21:	learn: 0.1304060	total: 488ms	remaining: 21.7s
22:	learn: 0.1303679	total: 513ms	remaining: 21.8s
23:	learn: 0.1303374	total: 538ms	remaining: 21.9s
24:	learn: 0.1303063	total: 560ms	remaining: 21.9s
25:	learn: 0.1302978	t

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=<catboost.core.CatBoostRegressor object at 0x7f2213637290>,
             param_grid={'eval_metric': ['RMSE'], 'learning_rate': [0.05, 0.01],
                         'max_depth': [6, 8, 10], 'n_estimators': [500, 1000]},
             return_train_score=True, scoring='r2', verbose=1000)

In [60]:
# kf=KFold(n_splits=5)
# score=cross_val_score(model,
#                       X_train,
#                       y_train,
#                       cv=kf,
#                       verbose=False)

In [61]:
# score

In [64]:
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_eval_metric,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,4.123592,0.109387,0.020066,0.001638,RMSE,0.05,6,500,"{'eval_metric': 'RMSE', 'learning_rate': 0.05,...",0.023483,...,0.017477,0.015893,6,0.101202,0.097869,0.115762,0.078820,0.096876,0.098106,0.011788
1,8.520608,0.139505,0.031576,0.004847,RMSE,0.05,6,1000,"{'eval_metric': 'RMSE', 'learning_rate': 0.05,...",0.029201,...,0.010996,0.021360,9,0.139596,0.144392,0.149514,0.113212,0.138026,0.136948,0.012529
2,6.463860,0.103044,0.024971,0.001154,RMSE,0.05,8,500,"{'eval_metric': 'RMSE', 'learning_rate': 0.05,...",0.026977,...,0.012413,0.023198,8,0.117772,0.127145,0.127398,0.101200,0.112661,0.117235,0.009796
3,13.553124,0.083906,0.046434,0.003229,RMSE,0.05,8,1000,"{'eval_metric': 'RMSE', 'learning_rate': 0.05,...",0.022291,...,0.000952,0.029630,11,0.170760,0.183705,0.180798,0.149150,0.168781,0.170639,0.012159
4,12.268564,0.274335,0.045635,0.012636,RMSE,0.05,10,500,"{'eval_metric': 'RMSE', 'learning_rate': 0.05,...",0.014327,...,0.008158,0.019454,10,0.133059,0.142354,0.148634,0.122287,0.134289,0.136125,0.008939
5,25.866053,0.454473,0.071297,0.006669,RMSE,0.05,10,1000,"{'eval_metric': 'RMSE', 'learning_rate': 0.05,...",-0.003326,...,-0.010064,0.022839,12,0.195059,0.213961,0.207166,0.179123,0.197111,0.198484,0.011861
6,4.370452,0.109388,0.023309,0.001700,RMSE,0.01,6,500,"{'eval_metric': 'RMSE', 'learning_rate': 0.01,...",0.022683,...,0.019668,0.006134,2,0.045145,0.040168,0.049135,0.034322,0.040969,0.041948,0.004983
7,8.311740,0.276532,0.035548,0.002649,RMSE,0.01,6,1000,"{'eval_metric': 'RMSE', 'learning_rate': 0.01,...",0.025163,...,0.020579,0.009914,1,0.059136,0.053611,0.066461,0.045054,0.054701,0.055793,0.007018
8,6.412665,0.125347,0.032374,0.005268,RMSE,0.01,8,500,"{'eval_metric': 'RMSE', 'learning_rate': 0.01,...",0.019817,...,0.019120,0.006312,4,0.049620,0.046973,0.057643,0.040143,0.048606,0.048597,0.005603
9,12.287998,0.285757,0.052663,0.003655,RMSE,0.01,8,1000,"{'eval_metric': 'RMSE', 'learning_rate': 0.01,...",0.023370,...,0.019470,0.010610,3,0.066947,0.064477,0.077297,0.054435,0.065513,0.065734,0.007273


In [62]:
y_pred = model_cv.predict(X_test)

In [63]:
rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print('Testing performance')
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2))
print('MAE: {:.2f}'.format(mae))

Testing performance
RMSE: 0.13
R2: 0.03
MAE: 0.06
